In [1]:
import sys, os
import pandas as pd
import numpy as np

# File dataset mẫu. Sửa đường dẫn thành đường dẫn đến file dataset của bạn.

DATASET_PATH = r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\not_remove_stopwords.xlsx"

Tiến hành đọc dữ liệu từ file

In [2]:
file : pd.DataFrame = pd.read_excel(DATASET_PATH)

# I. Sơ lược qua về tập huấn luyện:

ở đây tôi làm việc với tập huấn luyện của tôi. Do đó, khi làm việc với tập huấn luyện của bạn sẽ có một chút khác biệt. Tùy chỉnh theo tập huấn luyện của bạn.

In [3]:
file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2289 entries, 0 to 2288
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   lower      2289 non-null   object
 1   label      2289 non-null   object
 2   Join_word  2289 non-null   object
dtypes: object(3)
memory usage: 53.8+ KB


In [4]:
file.head()

,lower,label,Join_word
0,các em gái trường mình xinh quá,Positive,"['các', 'em_gái', 'trường', 'mình', 'xinh', 'q..."
1,có ai có đề khum ạ,Neutral,"['có', 'ai', 'có', 'đề', 'khum', 'ạ']"
2,17đ liệu đỗ vào kế toán của trường ko ạ khoa ...,Neutral,"['17đ', 'liệu', 'đỗ', 'vào', 'kế_toán', 'của',..."
3,mình cần tìm 1 bạn nữa học hộ 2 buổi vào tuần ...,Neutral,"['mình', 'cần', 'tìm', '1', 'bạn', 'nữa', 'học..."
4,ơ năm nay sv k được ăn món kẹo truyền thống nữ...,Neutral,"['ơ', 'năm', 'nay', 'sv', 'k', 'được', 'ăn', '..."


Loại bỏ cột lower (Không cần thiết)

In [5]:
file = file.drop(columns="lower")
file.head()

,label,Join_word
0,Positive,"['các', 'em_gái', 'trường', 'mình', 'xinh', 'q..."
1,Neutral,"['có', 'ai', 'có', 'đề', 'khum', 'ạ']"
2,Neutral,"['17đ', 'liệu', 'đỗ', 'vào', 'kế_toán', 'của',..."
3,Neutral,"['mình', 'cần', 'tìm', '1', 'bạn', 'nữa', 'học..."
4,Neutral,"['ơ', 'năm', 'nay', 'sv', 'k', 'được', 'ăn', '..."


Mô tả về tập dữ liệu:

In [7]:
file.describe()

,label,Join_word
count,2289,2289
unique,5,2276
top,Positive,"['vào', 'clb', 'để', 'nhận', 'bí', 'kíp', 'lấy..."
freq,1033,2


Loại bỏ các hàng missing value:

In [6]:
file = file.dropna()
file.describe()

,label,Join_word
count,2289,2289
unique,5,2276
top,Positive,"['vào', 'clb', 'để', 'nhận', 'bí', 'kíp', 'lấy..."
freq,1033,2


Loại bỏ các dòng trùng lặp

In [8]:
file = file.drop_duplicates(subset=['Join_word'])
file.describe()

,label,Join_word
count,2276,2276
unique,5,2276
top,Positive,"['các', 'em_gái', 'trường', 'mình', 'xinh', 'q..."
freq,1025,1


Tập huấn luyện bao gồm 2276 đối tượng. Mô tả về tập nhãn:

In [10]:
file.get("label").describe()

count         2276
unique           5
top       Positive
freq          1025
Name: label, dtype: object

Nhận thấy có hai nhãn bị sai lệch (sai bản gõ hoặc lower). Ta tiến hành xử lý chúng:

In [11]:
# Đưa ra danh sách các loại nhãn:

unique_labels: np.ndarray = file["label"].unique()
unique_labels.sort()
print(unique_labels)


['Negative' 'Neutral' 'Positive' 'Positive ' 'positive']


Nhận thấy có hai nhãn bị sai lệch nhưng do là cách gõ, nhưng giá trị thì là positive. Do đó chỉ cần chạy script để sửa lại:

In [12]:
# Xử lý nhãn sử dụng hàm apply
handle_labels = file["label"].apply(
    lambda label: label if label in unique_labels[0:3] else unique_labels[2])
file = file.drop(columns="label")
file["Labels"] = handle_labels


Kiểm tra lại

In [13]:
file["Labels"].unique()

array(['Positive', 'Neutral', 'Negative'], dtype=object)

Thực hiện reset index và trộn các nhãn với nhau

In [14]:
file = file.sample(frac=1).reset_index(drop=True)
file.head()

,Join_word,Labels
0,"['mình', 'cũng', 'đi', 'ngủ', 'mà', 'ngủ', 'ở'...",Neutral
1,"['tl', 'sai', '0', 'điểm', 'tp', 'k', 'tl', '0...",Neutral
2,"['may', 'là', 'đăng_kí', 'khảo_sát', 'đầu_vào']",Neutral
3,"['cô', 'an', 'thầy', 'đạt', 'thầy', 'tùng', 'a...",Positive
4,"['k62', 'được', 'chụp', 'lại', 'ảnh', 'thẻ', '...",Neutral


# II. Thực hiện dọn dẹp dữ liệu:

**Chú ý:** Do tôi đã có sẵn dữ liệu đã dọn dẹp nên phần II tôi xin được bỏ qua không chạy. Bạn đọc có thể tham khảo các bước dọn dẹp dữ liệu của tôi.


## 1. Thực hiện loại bỏ các ký tự đặc biệt, dấu câu

Các ký tự đặc biệt, dấu câu như: .,*&^%@!_-=+:;'"\}{}

In [20]:
import string
import re

string_pattele = string.punctuation + '\n'
string_pattele


def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

def remove_punctuation(text, except_char: str = None):
    try:
        result_text = "".join(
            char for char in text if char not in string_pattele or (except_char and char == except_char))
        result_text = remove_emoji(result_text)
    except TypeError:
        result_text = None
    return result_text

def filter_none_text_id(documents : pd.Series | list[str]) -> list[int]:
    need_filter : list[str] = [text for text in documents]
    none_text_index : list[int] = []
    for i, text in enumerate(need_filter):
        if (text is None):
            print(str(i) + ": " + str(text))
            none_text_index.append(i)
    
    return none_text_index

In [18]:
remove_punc : list[str] = []
for text in file["Opinion"]:
    remove_punc.append(remove_punctuation(text))

none_text_index = filter_none_text_id(remove_punc)

In [19]:
file['Remove Punc'] = remove_punc

In [20]:
# file = file.drop(none_text_index)
print(file.iloc[none_text_index])
file = file.drop(none_text_index).reset_index(drop=True)

Empty DataFrame
Columns: [Opinion, Labels, Remove Punc]
Index: []


In [22]:
file['Opinion'].iloc[353]

'Ko ngờ quản trị kinh doanh nghành mình học lại gần như điểm vào cao nhất trường'

## 2. Chuyển sang ký tự thường:

In [23]:
def lower_case(text):
    if text != None:
        return text.lower()

In [24]:
file['lower'] = file['Remove Punc'].apply(lambda x: lower_case(x))


In [25]:
file['lower'].head()

0                         trai xây dựng toàn trai xấu 
1                  cơ sở vật chất khang trang hiện đại
2    “ căng tin bánh mỳ 12k rẻ hơn ở ngoài 3k nhé  ...
3                         trg ngậm 100k chưa thấy bắn 
4    mình đi sớm tầm 12h10 rất rất vắng đỗ xe xếp n...
Name: lower, dtype: object

Kiểm tra xem còn có trùng lặp nữa hay không:

In [26]:
file.describe()

,Opinion,Labels,Remove Punc,lower
count,2306,2306,2306,2306
unique,2306,5,2297,2289
top,Trai xây dựng toàn trai xấu :((,Positive,Lớp mình còn không có gáiToàn vịt đực,utc mãi đỉnh
freq,1,1046,2,5


Có thể thấy sau khi xử lý 2 bước trên sinh ra 17 giá trị trùng lặp. Ta sẽ loại bỏ chúng

In [27]:
file = file.drop_duplicates(subset=['lower'])
file.describe()

,Opinion,Labels,Remove Punc,lower
count,2289,2289,2289,2289
unique,2289,5,2289,2289
top,Trai xây dựng toàn trai xấu :((,Positive,Trai xây dựng toàn trai xấu,trai xây dựng toàn trai xấu
freq,1,1033,1,1


Reset index

In [28]:
file = file.sample(frac=1).reset_index(drop=True)

In [31]:
file.head(20)

,Opinion,Labels,Remove Punc,lower
0,các em gái trường mình xinh quá,Positive,các em gái trường mình xinh quá,các em gái trường mình xinh quá
1,Có ai có đề khum ạ?,Neutral,Có ai có đề khum ạ,có ai có đề khum ạ
2,17đ liệu đỗ vào kế toán của trường ko ạ @@ kho...,Neutral,17đ liệu đỗ vào kế toán của trường ko ạ khoa ...,17đ liệu đỗ vào kế toán của trường ko ạ khoa ...
3,Mình cần tìm 1 bạn nữa học hộ 2 buổi vào tuần ...,Neutral,Mình cần tìm 1 bạn nữa học hộ 2 buổi vào tuần ...,mình cần tìm 1 bạn nữa học hộ 2 buổi vào tuần ...
4,ơ năm nay sv k được ăn món kẹo truyền thống nữ...,Neutral,ơ năm nay sv k được ăn món kẹo truyền thống nữ...,ơ năm nay sv k được ăn món kẹo truyền thống nữ...
5,là 1 điều tuỵt zời đấy em ạ,Positive,là 1 điều tuỵt zời đấy em ạ,là 1 điều tuỵt zời đấy em ạ
6,Mún đi học quá,Positive,Mún đi học quá,mún đi học quá
7,cơ sở vật chất tốt,Positive,cơ sở vật chất tốt,cơ sở vật chất tốt
8,Nhành dễ học nhất trong trường ra trường ko sợ...,Positive,Nhành dễ học nhất trong trường ra trường ko sợ...,nhành dễ học nhất trong trường ra trường ko sợ...
9,Có đi inova thì gọi tớ đi làm camera man với,Neutral,Có đi inova thì gọi tớ đi làm camera man với,có đi inova thì gọi tớ đi làm camera man với


In [38]:
file_out = file[['lower']].copy()
file_out['label'] = file['Labels']
file_out.head(20)

,lower,label
0,các em gái trường mình xinh quá,Positive
1,có ai có đề khum ạ,Neutral
2,17đ liệu đỗ vào kế toán của trường ko ạ khoa ...,Neutral
3,mình cần tìm 1 bạn nữa học hộ 2 buổi vào tuần ...,Neutral
4,ơ năm nay sv k được ăn món kẹo truyền thống nữ...,Neutral
5,là 1 điều tuỵt zời đấy em ạ,Positive
6,mún đi học quá,Positive
7,cơ sở vật chất tốt,Positive
8,nhành dễ học nhất trong trường ra trường ko sợ...,Positive
9,có đi inova thì gọi tớ đi làm camera man với,Neutral


In [39]:
with pd.ExcelWriter(os.path.join(os.getcwd(), r"dataset\lower.xlsx"), engine='xlsxwriter') as writer:
    file_out.to_excel(writer, sheet_name="Sheet1")


## 3. Tách từ tiếng Việt:

In [ ]:
from py_vncorenlp.vncorenlp import vncorenlp

text = "NGÀNH KỸ THUẬT ĐIỀU KHIỂN VÀ TỰ ĐỘNG HOÁ Mã ngành 7520216 Tổ hợp xét tuyển A00A01D07 Chỉ tiêu xét tuyển Kết quả thi THPT 190 Học bạ THPT 0 Thời gian đào tạo 5 năm Gồm 3 chuyên ngành Kỹ thuật điều khiển và tự động hoá giao thông Tự động hoá Mục tiêu đào tạo Đào tạo kĩ sư cử nhân “Kỹ thuật Điều khiển và Tự động hoá” có phương pháp tư duy sáng tạo kĩ năng thực hành và làm việc độc lập và làm việc nhóm Có kiến thức chuyên môn và kĩ năng mềm Đạt chuẩn tiếng anh B1 đảm bảo thích ứng với môi trường làm việc hiện đại Có thể tiếp tục học sau Đại học để được cấp bằng Thạc sĩ Tiến sĩ Kiến thức chuyên sâu về nhiều lĩnh vực chuyên ngànhVị trí công tác sau khi ra trường làCác công ty tập đoàn có trang bị hệ thống Điện  điện tử Tự động hoá Samsung Canon Toyota…Các công ty nhà máy cơ sở sản xuất trong Giao thông vận tảiCác tập đoàn công ty về lập trình điều khiểnCác công ty tư vấn và thiết kế thương mại kĩ thuật và chuyển giao công nghệ trong lĩnh vực Kĩ thuật điều khiển  tự động hoá và Giao thông vận tảiViện nghiên cứu trung tập công ty khoa học công nghệ các trường Đại học Cao đẳng Dạy nghề thuộc lĩnh vực Kĩ thuật điều khiển  Tự động hoá trong công nghiệp và Giao thông vận tảiBộ Khoa học công nghệ các sở KHCN bộ Giao thông vận tải sở Giao thông vận tải liên tỉnh các Ban quản lý dự án các Bộ Ngành liên quan"

# annotator_text = annotator.annotate(text)

word_segmented_text = annotator.tokenize(text)

## 4. Tokenization:
**Mục đích:** Chia nhỏ các câu thành các từ đơn vị nhỏ nhất.

In [22]:
def tokenization(text):
    word_list = text.split()
    token = list()
    for word in word_list:
        if word != "":
            token.append(word)
    return token


In [23]:
file['tokenization'] = file['lower'].apply(lambda x : tokenization(x))

file['tokenization'].head(15).to_list()


[['thầy', 'nam', '100điểm', 'luôn'],
 ['đây',
  'là',
  'trường',
  'mà',
  'tôi',
  'sẽ',
  'theo',
  'học',
  '4',
  'năm',
  'tự',
  'hào',
  'là',
  'sv',
  'giao',
  'thông'],
 ['gửi',
  'các',
  'em',
  'danh',
  'sách',
  'sinh',
  'viên',
  'không',
  'đủ',
  'không',
  'đủ',
  'kiện',
  'làm',
  'đồ',
  'án',
  'các',
  'em',
  'đã',
  'đăng',
  'ký',
  'thực',
  'hiện',
  'hủy',
  'học',
  'phần',
  'đồ',
  'án',
  'trả',
  'nợ',
  'các',
  'môn',
  'học',
  'để',
  'đủ',
  'điều',
  'kiện',
  'làm',
  'đồ',
  'án',
  'trong',
  'kỳ',
  'tới'],
 ['nhớ',
  'thời',
  'mua',
  'đề',
  'cương',
  'về',
  'ôn',
  'thi',
  'và',
  'kết',
  'quả',
  'là',
  'đc',
  'thi',
  'lại'],
 ['lúc',
  'mình',
  'nhập',
  'học',
  'chả',
  'được',
  'chụp',
  'ảnh',
  'thẻ',
  'sv',
  'gì',
  'cả'],
 ['chờ', 'đợi', 'các', 'pé', 'quá', 'nè'],
 ['nhiều',
  'quán',
  'ziệu',
  'để',
  'ae',
  'tỉ',
  'thí',
  'võ',
  'công',
  'xôi',
  'rẻ',
  'thuốc',
  'lào',
  'cổng',
  'trường',
  'hút',
  '

## 5. Loại bỏ từ dừng:

In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
stopword_eng_list = nltk.corpus.stopwords.words('english')

In [ ]:
vietnamese_stopword_file = r"E:\Learn Machine Learning\Project\Spam Email Classifier\train\vietnamese-stopwords.txt"
with open(vietnamese_stopword_file, encoding="utf-8") as file_vn: 
    stopword_vie_list = file_vn.read()




In [ ]:
stopword_vie_list = stopword_vie_list.split("\n")

In [ ]:
stopword_vie_list[:100]

In [ ]:
def remove_stopword(token):
    if token != None:
        result = list()
        for word in token:
            if (word not in stopword_eng_list) and (word not in stopword_vie_list):
                result.append(word)
        return result

In [ ]:
remove_stopword(tokenization(lower_case(remove_punctuation(
    "Năm đầu chơi thoải mái, năm 2 chịu khó chút để học lại ít thôi, năm 3 4 vất hơn, hạn chế bị học lại ko là bị đuổi đấy. Bạn a học đến năm cuối bị đuổi vì nợ môn nhiều quá"))))


In [ ]:
file['Remove stopword'] = file['tokenization'].apply(lambda x : remove_stopword(x))

In [ ]:
file['Remove stopword'].iloc[10]

**Note:** Hai bước sau chỉ thực hiện với tiếng Anh. Tiếng Việt không cần thiết.
## 6. Stemming

In [24]:
from nltk.stem.porter import PorterStemmer
#defining the object for stemming
porter_stemmer = PorterStemmer()
#defining a function for stemming
def stemming(text):
    stem_text = [porter_stemmer.stem(word) for word in text]
    return stem_text


In [ ]:
test_text = "Bài dễ thương quá aaa <3 hi vọng có nhiều người hơn nghe bài đáng iu này ^^"
stemming(remove_stopword(tokenization(lower_case(remove_punctuation(test_text)))))


## 7. Lemmatization:

In [ ]:
nltk.download('wordnet')


In [ ]:
nltk.download('omw-1.4')


In [ ]:
from nltk.stem import WordNetLemmatizer

class CustomLemmatizer(WordNetLemmatizer):
    def extract_lemmatizer(self, text):
        lemma_text = [self.lemmatize(word) for word in text]
        return lemma_text

cus_lemmatizer = CustomLemmatizer()

file['Lemmatizer'] = file['Remove stopword'].apply(lambda x : cus_lemmatizer.extract_lemmatizer(x))


In [ ]:
file['Lemmatizer'].head()


# 8. Xử lý trước khi đưa vào word embbedding

Copy một file mới từ file gốc. Mục đích để bảo tồn file gốc, nếu xảy ra vấn đề có thể sử dụng lại file gốc để tái tạo các bước.

In [22]:
handle_file = file.copy()
handle_file.head(20)


,Join_word,Labels
0,"['mình', 'cũng', 'đi', 'ngủ', 'mà', 'ngủ', 'ở'...",Neutral
1,"['tl', 'sai', '0', 'điểm', 'tp', 'k', 'tl', '0...",Neutral
2,"['may', 'là', 'đăng_kí', 'khảo_sát', 'đầu_vào']",Neutral
3,"['cô', 'an', 'thầy', 'đạt', 'thầy', 'tùng', 'a...",Positive
4,"['k62', 'được', 'chụp', 'lại', 'ảnh', 'thẻ', '...",Neutral
5,"['tiện', 'sale', 'thằng', 'bạn', 'nhé', 'nhà',...",Neutral
6,"['oh', 'thầy', 'dễ', 'nhá']",Positive
7,"['học', 'ở', 'đây', 'tốt']",Positive
8,"['học_phần', 'với', 'lớp_học_phần', 'là', 's_v...",Neutral
9,"['thí_sinh', 'này', 'vẫn', 'chép', 'tài_liệu',...",Neutral


Kiểm tra lần cuối xem có còn tồn tại dòng nào trống hay không. Dòng trống khiến việc chia trong TF-IDF không thể thực hiện, kéo theo lỗi trong mô hình

In [23]:
def check_zero_lenght_sentence(corpus : pd.Series | list[list[str]]):
    zero_len_docs : list[str] = []
    for index, doc in enumerate(corpus):
        if len(corpus) == 0:
            zero_len_docs.append(index)
    
    return zero_len_docs

Kết quả kiểm tra:

In [24]:
check_zero_lenght_sentence(handle_file["Join_word"])

[]

Loại bỏ toàn bộ các ký tự trừ kí tự '_' còn sót trong dữ liệu và tạo cột "Tokens" để lưu trữ:

In [25]:
handle_file["Tokens"] = handle_file["Join_word"].apply(lambda text: [remove_punctuation(x, '_').split()[0] for x in text.split(',')])

Drop cột "Join_word", tập dữ liệu đã sẵn sàng:

In [26]:
handle_file = handle_file.drop(columns="Join_word")
handle_file.head()

,Labels,Tokens
0,Neutral,"[mình, cũng, đi, ngủ, mà, ngủ, ở, bàn, trên, l..."
1,Neutral,"[tl, sai, 0, điểm, tp, k, tl, 0đ, tp, gọi, k, ..."
2,Neutral,"[may, là, đăng_kí, khảo_sát, đầu_vào]"
3,Positive,"[cô, an, thầy, đạt, thầy, tùng, ai, cũng, dễ, ..."
4,Neutral,"[k62, được, chụp, lại, ảnh, thẻ, không, ad]"


Tạo tập kho văn bản và tập nhãn tách rời. Tập kho văn bản gồm tổng hợp các văn bản trong cột "Tokens". Tập kho văn bản sẽ được nhúng TF-IDF, chuyển thành vector số. Sau đó, gắn tập vector với tập nhãn để cho ra tập dữ liệu có thể đưa vào huấn luyện mô hình.

In [22]:
corpus: list[list[str]] = [doc for doc in handle_file["Tokens"]]
labels: list[str] = handle_file["Labels"].to_list()


# III. TF-IDF:

**Công thức:** (Sử dụng công thức của thư viện scikit learn. Tuy nhiên có nhiều công thức khác nhau cho hai giá trị này)

* TF = f(t, d) = (số lần xuất hiện từ t trong văn bản d)

* IDF = ln((1 + len(D)) / (1 + len{d | t thuộc d})) + 1 = ln((1 + độ dài kho văn bản) / (1 + số lượng văn bản chứa từ t)) + 1

* TF-IDF = TF * IDF

**Chú ý:** để tránh mẫu số bằng 0, ta cộng 1 vào mẫu và tính tích TF-IDF bình thường

* IDF = ln(len(D) / (1 + len{d | t thuộc d}))

In [23]:
import math
def tf(document : list[str], term : str) -> float:
    tf_score = 0
    for words in document:
        if term == words:
            tf_score += 1
    return tf_score

def idf(corpus : list[list[str]], term : str) -> float:
    document_contain_term : int = 0
    for document in corpus:
        if term in document:
            document_contain_term += 1
    idf_score = float((1 + len(corpus)) / (document_contain_term + 1))
    return math.log(idf_score) + 1

def tf_idf(corpus : list[list[str]], document : list[str], term : str) -> float:
    return tf(document, term) * idf(corpus, term)

def features_extraction(corpus: list[list[str]]) -> set[str]:
    features: set = set()
    for document in corpus:
        for term in document:
            features.add(term)
    return features


def count_vectorizer(corpus: list[list[str]], vocabulary: list[str], fq: bool = True) -> list[list[int]]:
    vector: list[list[int]] = []
    for sen in corpus:
        sen_count: list[int] = []
        for term in vocabulary:
            sen_count.append(tf(sen, term) if fq else (
                1 if term in sen else 0))
        vector.append(sen_count)
    return vector


def tf_idf_vectorizer(corpus: list[list[str]], vocabulary : list[str]) -> list[dict[str, float]]:
    tf_idf_dict: list[dict[str, float]] = []

    for sen in corpus:
        tf_idf_sen: dict[str, float] = {}
        for term in sen:
            tf_idf_sen[term] = tf_idf(corpus, sen, term)
        tf_idf_dict.append(tf_idf_sen)

    tf_idf_corpus: list[list[float]] = []

    for sentence in tf_idf_dict:
        tf_idf_sentence: list[float] = []
        for term in vocabulary:
            tf_idf_sentence.append(sentence.get(term, 0))
        tf_idf_corpus.append(tf_idf_sentence)

    return tf_idf_corpus
# def word_vectorizer(document : list[str], features : set[str]) -> list[int]


Thử nghiệm với một kho văn bản đơn giản với 10 câu:

In [111]:
example_corpus = [
    "The cat sat on the mat.",
    "The dog chased its tail.",
    "The sun was shining brightly.",
    "The bird flew high in the sky.",
    "The flowers were blooming beautifully.",
    "The leaves rustled in the wind.",
    "The river flowed peacefully.",
    "The fish swam in the pond.",
    "The butterfly landed on a flower.",
    "The grass was green and lush."
]
example_corpus = [tokenization(lower_case(remove_punctuation(sens))) for sens in example_corpus]
example_corpus

[['the', 'cat', 'sat', 'on', 'the', 'mat'],
 ['the', 'dog', 'chased', 'its', 'tail'],
 ['the', 'sun', 'was', 'shining', 'brightly'],
 ['the', 'bird', 'flew', 'high', 'in', 'the', 'sky'],
 ['the', 'flowers', 'were', 'blooming', 'beautifully'],
 ['the', 'leaves', 'rustled', 'in', 'the', 'wind'],
 ['the', 'river', 'flowed', 'peacefully'],
 ['the', 'fish', 'swam', 'in', 'the', 'pond'],
 ['the', 'butterfly', 'landed', 'on', 'a', 'flower'],
 ['the', 'grass', 'was', 'green', 'and', 'lush']]

Sử dụng các hàm đã thiết lập để tính trọng số tf-idf

In [126]:

# Trích xuất thuộc tính cho ma trận tf-idf
features_example : list[str] = list(features_extraction(example_corpus))
features_example.sort()

# Tính trọng số tf-idf của từng từ trong 1 câu
tf_idf_example_vector = tf_idf_vectorizer(example_corpus, features_example)


In [135]:
pd.DataFrame(tf_idf_example_vector, columns=features_example)

,a,and,beautifully,bird,blooming,brightly,butterfly,cat,chased,dog,...,sat,shining,sky,sun,swam,tail,the,was,were,wind
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.704748,0.000000,0.000000,...,2.704748,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.704748,2.704748,...,0.000000,0.000000,0.000000,0.000000,0.000000,2.704748,1.0,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,2.704748,0.000000,0.000000,0.000000,0.000000,...,0.000000,2.704748,0.000000,2.704748,0.000000,0.000000,1.0,2.299283,0.000000,0.000000
3,0.000000,0.000000,0.000000,2.704748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.704748,0.000000,0.000000,0.000000,2.0,0.000000,0.000000,0.000000
4,0.000000,0.000000,2.704748,0.000000,2.704748,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,2.704748,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.0,0.000000,0.000000,2.704748
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,2.704748,0.000000,2.0,0.000000,0.000000,0.000000
8,2.704748,0.000000,0.000000,0.000000,0.000000,0.000000,2.704748,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000
9,0.000000,2.704748,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,2.299283,0.000000,0.000000


Sử dụng thư viện scikit learn để thử lại:

In [131]:
from sklearn.feature_extraction.text import TfidfTransformer

count_example: np.ndarray = np.array(count_vectorizer(
    corpus=example_corpus, vocabulary=features_example, fq=True), dtype=int)
count_example

t = TfidfTransformer(norm=None, use_idf=True, smooth_idf=True, sublinear_tf=False)
x = t.fit_transform(count_example)
x.toarray()


array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 2.70474809, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 2.70474809, 2.29928298,
        0.        , 0.        , 0.        , 0.        , 2.70474809,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        2.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 2.70474809, 2.70474809,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 2.70474809,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 2.7

Áp dụng với bộ giá trị của ta:

In [24]:
features = features_extraction(corpus=corpus)
tf_idf_vector = tf_idf_vectorizer(corpus=corpus, vocabulary=features)

In [25]:
features

{'máu_đơn',
 'trốn',
 'hiến',
 'tham_gia',
 'part',
 'từ_khoá',
 'đky',
 'ạ',
 'cay',
 'phục_vụ_việc',
 'dũng_sĩ',
 '99',
 'bắt_đầu',
 'thời_kì',
 'tuyệttttt',
 'bông',
 'tq',
 'website',
 'trườ',
 'gunners',
 'tin_yêu',
 'tìm_kiếm',
 'danh',
 'quyền_lợi',
 'nghị_định_thư',
 'cầu_lông',
 'nhéđi',
 'ute',
 'gia_nhập',
 'report',
 'cấp',
 'khát_vọng',
 'tuyệt',
 'lương_tâm',
 'ban_đầu',
 'khoa_học_sinh_viên',
 'update',
 'share',
 'nhiều5',
 'vặt',
 'ảo',
 'zô',
 'ợ',
 'chính_trị',
 'vận_dụng',
 'zữ',
 'chọn',
 'phụ_cấp',
 'cơ_khí',
 'zay',
 'chái',
 'đạo_đức',
 'tán',
 '54',
 'giây',
 'k56',
 'sân',
 '21ha',
 'nét',
 '2010',
 'chán',
 'ưu',
 'ak',
 'đẹp_đẽ',
 '10ml',
 'khá',
 'khao',
 'điểm_chuẩn',
 'tivi',
 'nhân_văn',
 'yêu_thương',
 'giảithí',
 'đgcòn',
 'ngọt',
 'thành_phần_nào',
 'kẹt',
 'cámà',
 'tự_động',
 'nền',
 'chu_đáo',
 'đầu_cơ_bản',
 'barca',
 'fai',
 'hình_như',
 'ngừoi',
 'thi_cử',
 'mún',
 'đứt',
 'ngầm',
 'cày',
 'so',
 '0978226816',
 'hay',
 '281',
 '16h',
 'vb2',
 'o

In [26]:
df = pd.DataFrame(tf_idf_vector, columns=list(features))
df["label"] = labels

In [27]:
df.head()

,máu_đơn,trốn,hiến,tham_gia,part,từ_khoá,đky,ạ,cay,phục_vụ_việc,...,quạo,công_chính,36,thành_đoàn,hà,ngắn,”,giải_lao,đàn_bà,label
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,Neutral
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,Negative
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,Positive
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,Positive
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.39841,0.0,0.0,Neutral


In [28]:
import os
os.getcwd()

'e:\\Learn Machine Learning\\Project\\Opinion Classifier\\res'

Lưu thành file csv

In [ ]:
df.to_csv(r"E:\Learn Machine Learning\Project\Opinion Classifier\res\dataset\tf-idf-not-remv-stopwords.csv")


Lưu thành file excel

In [62]:
with pd.ExcelWriter(os.path.join(os.getcwd(), r"dataset\tf-idf-df-not-remv-stopwords.xlsx"), engine='xlsxwriter') as writer:
    df.to_excel(writer, sheet_name="Sheet1")


Tạo một file lưu lại điểm số IDF của từng từ. Giá trị này sử dụng khi có dữ liệu mới được nạp vào, ta có thể tính được giá trị TF-IDF của văn bản mới:

In [ ]:
idf_term = {}

for term in features:
    idf_term[term] = idf(corpus, term)

import pickle

with open("idf_term.pkl", "wb") as f:
    pickle.dump(idf_term, f)